In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/poem-vn/new_data_clean2.csv
/kaggle/input/poem-vn/start_vowels.txt
/kaggle/input/poem-vn/tone_dict.txt
/kaggle/input/poem-vn/rhymes.txt
/kaggle/input/poem-vn/new_data_clean.csv


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from typing import List, Dict
import os

# # Đọc dữ liệu
# df = pd.read_csv('/kaggle/input/poem-vn/new_data_clean2.csv').head(10000)
# df

# Transformer

In [ ]:
import torch
import torch.nn as nn
import math
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tokenizers import Tokenizer, models, trainers, pre_tokenizers
import numpy as np
from typing import List
import re
from tqdm import tqdm  # Import tqdm

def is_vietnamese_word(word):
    vietnamese_pattern = r'^[aàảãáạăằẳẵắặâầẩẫấậbcdđeèẻẽéẹêềểễếệfghiìỉĩíịjklmnoòỏõóọôồổỗốộơờởỡớợpqrstuùủũúụưừửữứựvwxyỳỷỹýỵz]+$'
    return bool(re.match(vietnamese_pattern, word.lower()))

def get_rhyme(word):
    if not word or not is_vietnamese_word(word):
        return ''
    # Lấy âm cuối của từ
    vowels = 'aàảãáạăằẳẵắặâầẩẫấậeèẻẽéẹêềểễếệiìỉĩíịoòỏõóọôồổỗốộơờởỡớợuùủũúụưừửữứựyỳỷỹýỵ'
    word = word.lower()
    rhyme = ''
    for i in range(len(word)-1, -1, -1):
        if word[i] in vowels:
            rhyme = word[i:]
            break
    return rhyme

def get_tone(word):
    if not word or not is_vietnamese_word(word):
        return 'neutral'
    
    even_tones = 'aăâeêioôơuưy' + 'àằầèềìòồờùừỳ' + 'ảẳẩẻểỉỏổởủửỷ'
    uneven_tones = 'áắấéếíóốớúứý' + 'ạặậẹệịọộợụựỵ' + 'ãẵẫẽễĩõỗỡũữỹ'
    
    word = word.lower()
    last_char = word[-1]
    
    if last_char in even_tones:
        return 'even'
    elif last_char in uneven_tones:
        return 'uneven'
    return 'neutral'

class LucBatDataset(Dataset):
    def __init__(self, texts: List[str], tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode(text)
        
        input_ids = encoding.ids[:-1]
        labels = encoding.ids[1:]
        
        padding_length = self.max_length - len(input_ids)
        if padding_length > 0:
            input_ids = input_ids + [self.tokenizer.token_to_id("[PAD]")] * padding_length
            labels = labels + [self.tokenizer.token_to_id("[PAD]")] * padding_length
        
        return {
            "input_ids": torch.tensor(input_ids[:self.max_length]),
            "labels": torch.tensor(labels[:self.max_length])
        }

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class LucBatTransformer(nn.Module):
    def __init__(self, vocab_size: int, d_model: int = 512, nhead: int = 8, 
                 num_layers: int = 6, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, 
                                                  dim_feedforward=2048, 
                                                  dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        self.fc_out = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.embedding(src) * math.sqrt(self.d_model)
        embedded = self.pos_encoder(embedded)
        output = self.transformer_encoder(embedded)
        output = self.dropout(output)
        output = self.fc_out(output)
        return output

def create_tokenizer(texts: List[str]) -> Tokenizer:
    tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
    
    special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "\n"]
    trainer = trainers.WordPieceTrainer(
        vocab_size=12258, # láy trong token 
        special_tokens=special_tokens
    )
    
    tokenizer.train_from_iterator(texts, trainer=trainer)
    return tokenizer

def train_epoch(model: nn.Module, dataloader: DataLoader, 
                optimizer: torch.optim.Optimizer, device: torch.device):
    model.train()
    total_loss = 0
    
    # Wrap DataLoader with tqdm to show progress
    for batch in tqdm(dataloader, desc="Training", unit="batch"):  # Add tqdm here
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids)
        outputs = outputs.view(-1, outputs.size(-1))
        labels = labels.view(-1)
        
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

def get_tone_type(word):
    """Phân loại thanh điệu của từ"""
    vowels = {
        'even': ['a', 'ă', 'â', 'e', 'ê', 'o', 'ô', 'ơ', 'i', 'u', 'ư', 'y',
                'à', 'ằ', 'ầ', 'è', 'ề', 'ò', 'ồ', 'ờ', 'ì', 'ù', 'ừ', 'ỳ'],
        'uneven': ['á', 'ắ', 'ấ', 'é', 'ế', 'ó', 'ố', 'ớ', 'í', 'ú', 'ứ', 'ý',
                   'ạ', 'ặ', 'ậ', 'ẹ', 'ệ', 'ọ', 'ộ', 'ợ', 'ị', 'ụ', 'ự', 'ỵ',
                   'ả', 'ẳ', 'ẩ', 'ẻ', 'ể', 'ỏ', 'ổ', 'ở', 'ỉ', 'ủ', 'ử', 'ỷ',
                   'ã', 'ẵ', 'ẫ', 'ẽ', 'ễ', 'õ', 'ỗ', 'ỡ', 'ĩ', 'ũ', 'ữ', 'ỹ']
    }
    
    for char in word.lower():
        if char in vowels['even']:
            return 'even'
        elif char in vowels['uneven']:
            return 'uneven'
    return 'even'  # mặc định nếu không tìm thấy

def get_specific_tone(word):
    """Phân loại thanh điệu cụ thể (ngang/huyền)"""
    huyen_vowels = ['à', 'ằ', 'ầ', 'è', 'ề', 'ò', 'ồ', 'ờ', 'ì', 'ù', 'ừ', 'ỳ']
    khong_dau_vowels = ['a', 'ă', 'â', 'e', 'ê', 'o', 'ô', 'ơ', 'i', 'u', 'ư', 'y']
    
    for char in word.lower():
        if char in huyen_vowels:
            return 'huyen'
        elif char in khong_dau_vowels:
            return 'ngang'
    return None

def check_rhyme(word1, word2):
    """Kiểm tra vần của hai từ"""
    # Implement rhyme checking logic here
    # This is a simplified version
    return word1[-1] == word2[-1]

def generate_lucbat(model, tokenizer, prompt, temperature=0.7, device='cpu'):
    def get_valid_word(candidates, position, line_type, prev_line=None, prev_rhyme=None):
        # Quy tắc thanh điệu cho câu lục và câu bát
        tone_rules = {
            'luc': {
                2: 'even',    # Chữ 2: thanh bằng
                4: 'uneven',  # Chữ 4: thanh trắc
                6: 'even'     # Chữ 6: thanh bằng
            },
            'bat': {
                2: 'even',    # Chữ 2: thanh bằng
                4: 'uneven',  # Chữ 4: thanh trắc
                6: 'even',    # Chữ 6: thanh bằng
                8: 'even'     # Chữ 8: thanh bằng
            }
        }
        
        for word in candidates:
            if not isinstance(word, str) or len(word.strip()) == 0:
                continue
                
            word_tone = get_tone_type(word)
            
            # Kiểm tra quy tắc thanh điệu theo vị trí
            if position in tone_rules[line_type]:
                if word_tone != tone_rules[line_type][position]:
                    continue
            
            # Kiểm tra quy tắc vần
            if line_type == 'luc' and position == 6:
                # Chữ cuối câu lục phải vần với chữ 6 câu bát tiếp theo
                if prev_rhyme and not check_rhyme(word, prev_rhyme):
                    continue
                    
            if line_type == 'bat':
                if position == 6:
                    # Chữ 6 câu bát phải vần với chữ cuối câu lục trước
                    if prev_line and not check_rhyme(word, prev_line.split()[-1]):
                        continue
                elif position == 8:
                    # Quy tắc đặc biệt cho chữ 6 và 8 của câu bát
                    word6 = prev_line.split()[5] if prev_line else None
                    if word6:
                        word6_tone = get_specific_tone(word6)
                        word8_tone = get_specific_tone(word)
                        
                        # Nếu chữ 6 thanh ngang thì chữ 8 phải thanh huyền và ngược lại
                        if word6_tone == 'ngang' and word8_tone != 'huyen':
                            continue
                        if word6_tone == 'huyen' and word8_tone != 'ngang':
                            continue
            
            return word
        
        return candidates[0]  # Trường hợp không tìm được từ phù hợp

    model.eval()
    lines = []
    current_line = prompt.split()
    prev_rhyme = None
    
    # Sinh 4 dòng thơ
    for i in range(4):
        line_type = 'bat' if i % 2 == 1 else 'luc'
        target_length = 8 if line_type == 'bat' else 6
        
        while len(current_line) < target_length:
            input_text = ' '.join(lines + [' '.join(current_line)])
            input_ids = torch.tensor(tokenizer.encode(input_text).ids).unsqueeze(0).to(device)
            
            with torch.no_grad():
                outputs = model(input_ids)
                next_token_logits = outputs[0, -1, :] / temperature
                
                # Lấy nhiều candidates hơn để có nhiều lựa chọn
                top_k = 100
                top_k_logits, top_k_indices = torch.topk(next_token_logits, top_k)
                probs = torch.softmax(top_k_logits, dim=-1)
                
                candidate_indices = top_k_indices[torch.multinomial(probs, num_samples=20)]
                candidates = [tokenizer.decode([idx.item()]).strip() for idx in candidate_indices]
                
                next_word = get_valid_word(
                    candidates,
                    len(current_line) + 1,
                    line_type,
                    prev_line=' '.join(current_line) if current_line else None,
                    prev_rhyme=prev_rhyme
                )
                
                if next_word:
                    current_line.append(next_word)
        
        lines.append(' '.join(current_line))
        if line_type == 'bat':
            prev_rhyme = current_line[5]  # Lưu chữ thứ 6 của câu bát
        current_line = []
    
    return '\n'.join(lines)

def check_rhyme(word1, word2):
    rhyme1 = get_rhyme(word1)
    rhyme2 = get_rhyme(word2)
    return rhyme1 == rhyme2

def load_and_preprocess_data(file_path: str) -> List[str]:
    df = pd.read_csv(file_path)
    return df['content'].tolist()

def setup_training(texts: List[str], device: torch.device):
    tokenizer = create_tokenizer(texts)
    dataset = LucBatDataset(texts, tokenizer)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    
    model = LucBatTransformer(
        vocab_size=tokenizer.get_vocab_size(),
        d_model=512,
        nhead=8,
        num_layers=6
    ).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    return model, tokenizer, dataloader, optimizer

def train_model(texts: List[str], num_epochs: int = 20):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model, tokenizer, dataloader, optimizer = setup_training(texts, device)
    
    for epoch in range(num_epochs):
        loss = train_epoch(model, dataloader, optimizer, device)
        print(f'Epoch {epoch+1}, Loss: {loss:.4f}')
    
    return model, tokenizer

def generate_poem(model, tokenizer, prompt: str, num_stanzas: int = 2):
    device = next(model.parameters()).device
    poem = generate_lucbat(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        temperature=0.8,
        device=device,
        num_stanzas=num_stanzas
    )
    return poem

In [25]:
tokenizer

Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[{"id":0, "content":"[PAD]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":1, "content":"[UNK]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":2, "content":"[CLS]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":3, "content":"[SEP]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":4, "content":"[MASK]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":5, "content":"
", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}], normalizer=None, pre_tokenizer=Whitespace(), post_processor=None, decoder=None, model=WordPiece(unk_token="[UNK]", continuing_subword_prefix="##", max_input_chars_per_word=100, vocab={"[PAD]":0, "[UNK]":1, "[CLS]":2, "[SEP]"

In [26]:
# Đầu tiên load data và train model
texts = load_and_preprocess_data('/kaggle/input/poem-vn/new_data_clean2.csv')
model, tokenizer = train_model(texts, num_epochs=20)

# Save the tokenizer using the save method from the tokenizers library
tokenizer.save("/kaggle/working/tokenizer.json")
torch.save(model.state_dict(), "/kaggle/working/lucbat_model.pth")

# Định nghĩa device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Generating
prompt = "thăm con ở trại"
poem = generate_lucbat(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    temperature=0.7,
    device=device
)
print("\nBài thơ được tạo:")
print(poem)


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Training: 100%|██████████| 15668/15668 [29:33<00:00,  8.83batch/s]


Epoch 1, Loss: 2.1006


Training: 100%|██████████| 15668/15668 [29:24<00:00,  8.88batch/s]


Epoch 2, Loss: 2.1120


Training: 100%|██████████| 15668/15668 [29:24<00:00,  8.88batch/s]


...

Training: 100%|██████████| 15668/15668 [29:21<00:00,  8.90batch/s]


Epoch 19, Loss: 2.1101


Training: 100%|██████████| 15668/15668 [29:22<00:00,  8.89batch/s]

Epoch 20, Loss: 2.1101


AttributeError: 'tokenizers.Tokenizer' object has no attribute 'save_model'

=== Kích thước của mô hình ===  
Vocabulary Size: 12,258 tokens  
Embedding Dimension (d_model): 512  
Number of Attention Heads: 8  
Number of Layers: 6  
Feed Forward Dimension: 2048  
Dropout Rate: 0.1  

Tổng số tham số: 31,478,754  

=== Kích thước đầu vào ===
Input shape: [batch_size, sequence_length]
- Batch size: 16 (mặc định)
- Max sequence length: 128 tokens

Embedding output shape: [batch_size, sequence_length, d_model]
- [16, 128, 512]


In [27]:
# Định nghĩa device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Generating
prompt = "thăm con ở trại"
poem = generate_lucbat(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    temperature=0.7,
    device=device
)
print("\nBài thơ được tạo:")
print(poem)


Bài thơ được tạo:
thăm con ở trại người duyên
mây yêu nhớ nhớ như như trời vào
tình đời ta một mình như
đã đi người mắt có gió chiều về


In [33]:
# Định nghĩa device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Generating
prompt = "anh đi anh nhớ quê nhà"
poem = generate_lucbat(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    temperature=0.7,
    device=device
)
print("\nBài thơ được tạo:")
print(poem)


Bài thơ được tạo:
anh đi anh nhớ quê nhà
lòng trong người để ngày ngày tiếng sao
em xuân yêu để nắng ngày
như không biết lại anh anh câu ngày


In [3]:
import torch
import torch.nn as nn
import math
from tokenizers import Tokenizer
import re

# Định nghĩa lại các class và function cần thiết
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class LucBatTransformer(nn.Module):
    def __init__(self, vocab_size: int, d_model: int = 512, nhead: int = 8, 
                 num_layers: int = 6, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, 
                                                  dim_feedforward=2048, 
                                                  dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        self.fc_out = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.embedding(src) * math.sqrt(self.d_model)
        embedded = self.pos_encoder(embedded)
        output = self.transformer_encoder(embedded)
        output = self.dropout(output)
        output = self.fc_out(output)
        return output

def get_rhyme(word):
    if not word or not is_vietnamese_word(word):
        return ''
    vowels = 'aàảãáạăằẳẵắặâầẩẫấậeèẻẽéẹêềểễếệiìỉĩíịoòỏõóọôồổỗốộơờởỡớợuùủũúụưừửữứựyỳỷỹýỵ'
    word = word.lower()
    rhyme = ''
    for i in range(len(word)-1, -1, -1):
        if word[i] in vowels:
            rhyme = word[i:]
            break
    return rhyme

def is_vietnamese_word(word):
    vietnamese_pattern = r'^[aàảãáạăằẳẵắặâầẩẫấậbcdđeèẻẽéẹêềểễếệfghiìỉĩíịjklmnoòỏõóọôồổỗốộơờởỡớợpqrstuùủũúụưừửữứựvwxyỳỷỹýỵz]+$'
    return bool(re.match(vietnamese_pattern, word.lower()))

def get_tone_type(word):
    vowels = {
        'even': ['a', 'ă', 'â', 'e', 'ê', 'o', 'ô', 'ơ', 'i', 'u', 'ư', 'y',
                'à', 'ằ', 'ầ', 'è', 'ề', 'ò', 'ồ', 'ờ', 'ì', 'ù', 'ừ', 'ỳ'],
        'uneven': ['á', 'ắ', 'ấ', 'é', 'ế', 'ó', 'ố', 'ớ', 'í', 'ú', 'ứ', 'ý',
                   'ạ', 'ặ', 'ậ', 'ẹ', 'ệ', 'ọ', 'ộ', 'ợ', 'ị', 'ụ', 'ự', 'ỵ',
                   'ả', 'ẳ', 'ẩ', 'ẻ', 'ể', 'ỏ', 'ổ', 'ở', 'ỉ', 'ủ', 'ử', 'ỷ',
                   'ã', 'ẵ', 'ẫ', 'ẽ', 'ễ', 'õ', 'ỗ', 'ỡ', 'ĩ', 'ũ', 'ữ', 'ỹ']
    }
    
    for char in word.lower():
        if char in vowels['even']:
            return 'even'
        elif char in vowels['uneven']:
            return 'uneven'
    return 'even'

def get_specific_tone(word):
    huyen_vowels = ['à', 'ằ', 'ầ', 'è', 'ề', 'ò', 'ồ', 'ờ', 'ì', 'ù', 'ừ', 'ỳ']
    khong_dau_vowels = ['a', 'ă', 'â', 'e', 'ê', 'o', 'ô', 'ơ', 'i', 'u', 'ư', 'y']
    
    for char in word.lower():
        if char in huyen_vowels:
            return 'huyen'
        elif char in khong_dau_vowels:
            return 'ngang'
    return None

def check_rhyme(word1, word2):
    rhyme1 = get_rhyme(word1)
    rhyme2 = get_rhyme(word2)
    return rhyme1 == rhyme2

def generate_lucbat(model, tokenizer, prompt, temperature=0.7, device='cpu'):
    def get_valid_word(candidates, position, line_type, prev_line=None, prev_rhyme=None):
        # Quy tắc thanh điệu cho câu lục và câu bát
        tone_rules = {
            'luc': {
                2: 'even',    # Chữ 2: thanh bằng
                4: 'uneven',  # Chữ 4: thanh trắc
                6: 'even'     # Chữ 6: thanh bằng
            },
            'bat': {
                2: 'even',    # Chữ 2: thanh bằng
                4: 'uneven',  # Chữ 4: thanh trắc
                6: 'even',    # Chữ 6: thanh bằng
                8: 'even'     # Chữ 8: thanh bằng
            }
        }
        
        for word in candidates:
            if not isinstance(word, str) or len(word.strip()) == 0:
                continue
                
            word_tone = get_tone_type(word)
            
            # Kiểm tra quy tắc thanh điệu theo vị trí
            if position in tone_rules[line_type]:
                if word_tone != tone_rules[line_type][position]:
                    continue
            
            # Kiểm tra quy tắc vần
            if line_type == 'luc' and position == 6:
                # Chữ cuối câu lục phải vần với chữ 6 câu bát tiếp theo
                if prev_rhyme and not check_rhyme(word, prev_rhyme):
                    continue
                    
            if line_type == 'bat':
                if position == 6:
                    # Chữ 6 câu bát phải vần với chữ cuối câu lục trước
                    if prev_line and not check_rhyme(word, prev_line.split()[-1]):
                        continue
                elif position == 8:
                    # Quy tắc đặc biệt cho chữ 6 và 8 của câu bát
                    word6 = prev_line.split()[5] if prev_line else None
                    if word6:
                        word6_tone = get_specific_tone(word6)
                        word8_tone = get_specific_tone(word)
                        
                        # Nếu chữ 6 thanh ngang thì chữ 8 phải thanh huyền và ngược lại
                        if word6_tone == 'ngang' and word8_tone != 'huyen':
                            continue
                        if word6_tone == 'huyen' and word8_tone != 'ngang':
                            continue
            
            return word
        
        return candidates[0]  # Trường hợp không tìm được từ phù hợp

    model.eval()
    lines = []
    current_line = prompt.split()
    prev_rhyme = None
    
    # Sinh 4 dòng thơ
    for i in range(4):
        line_type = 'bat' if i % 2 == 1 else 'luc'
        target_length = 8 if line_type == 'bat' else 6
        
        while len(current_line) < target_length:
            input_text = ' '.join(lines + [' '.join(current_line)])
            input_ids = torch.tensor(tokenizer.encode(input_text).ids).unsqueeze(0).to(device)
            
            with torch.no_grad():
                outputs = model(input_ids)
                next_token_logits = outputs[0, -1, :] / temperature
                
                # Lấy nhiều candidates hơn để có nhiều lựa chọn
                top_k = 100
                top_k_logits, top_k_indices = torch.topk(next_token_logits, top_k)
                probs = torch.softmax(top_k_logits, dim=-1)
                
                candidate_indices = top_k_indices[torch.multinomial(probs, num_samples=20)]
                candidates = [tokenizer.decode([idx.item()]).strip() for idx in candidate_indices]
                
                next_word = get_valid_word(
                    candidates,
                    len(current_line) + 1,
                    line_type,
                    prev_line=' '.join(current_line) if current_line else None,
                    prev_rhyme=prev_rhyme
                )
                
                if next_word:
                    current_line.append(next_word)
        
        lines.append(' '.join(current_line))
        if line_type == 'bat':
            prev_rhyme = current_line[5]  # Lưu chữ thứ 6 của câu bát
        current_line = []
    
    return '\n'.join(lines)

# Hàm chính để load model và sinh thơ
def load_model_and_generate(model_path, tokenizer_path, prompt):
    # Load tokenizer
    tokenizer = Tokenizer.from_file(tokenizer_path)
    
    # Khởi tạo model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = LucBatTransformer(
        vocab_size=tokenizer.get_vocab_size(),
        d_model=512,
        nhead=8,
        num_layers=6
    ).to(device)
    
    # Load model weights
    model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
    model.eval()
    
    # Sinh thơ
    poem = generate_lucbat(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        temperature=0.8,
        device=device
    )
    
    return poem

# Sử dụng
if __name__ == "__main__":
    model_path = "/kaggle/input/poem-vn/lucbat_model.pth"  # Đường dẫn tới file model đã lưu
    tokenizer_path = "/kaggle/input/poem-vn/tokenizer.json"  # Đường dẫn tới file tokenizer đã lưu
    prompt = "Mùa xuân"  # Prompt để bắt đầu bài thơ
    
    poem = load_model_and_generate(model_path, tokenizer_path, prompt)
    print(poem)


Mùa xuân ta một là thương
chẳng còn như đến như như năm trời
hương trăng em nhớ hồng như
để lòng đi có em em chờ còn
